# Amazon Orders from SP-API

In [19]:
import pandas as pd 
import pymongo 
from jsonpath import jsonpath

## Filters
- {"items.OrderItems.QuantityOrdered": { $gt: 1 }}
- {$where: "this.items.OrderItems.length > 1"}   304-3051997-3024347
- {"ShippingPrice": { $exists: false }}  305-9788680-9105155
- {"order.OrderStatus": "Unshipped"}
- {"order.PurchaseDate": { $regex: "^2024-04-14"}}

In [27]:
db_client = pymongo.MongoClient("192.168.8.10", 27017)
db_names = db_client.list_database_names()
print(db_names)
orders_collection = db_client['amazon_data']["orders"]
# Filters
#  "306-4644186-6279565" # 306-7409173-7592316 302-0196774-9621929  306-4644186-6279565 304-3051997-3024347
# order = orders_collection.find_one({"_id": "305-9788680-9105155"}, )
order = orders_collection.find_one({"_id": "304-3051997-3024347"}, )

print(order)

['admin', 'amazon_data', 'config', 'local']
{'_id': '304-3051997-3024347', 'fetchedAt': '2024-04-21T01:21:06Z', 'items': {'OrderItems': [{'ProductInfo': {'NumberOfItems': '100'}, 'BuyerInfo': {}, 'ItemTax': {'CurrencyCode': 'EUR', 'Amount': '3.20'}, 'QuantityShipped': 2, 'BuyerRequestedCancel': {'IsBuyerRequestedCancel': 'false', 'BuyerCancelReason': ''}, 'ItemPrice': {'CurrencyCode': 'EUR', 'Amount': '19.98'}, 'ASIN': 'B0CS6C67XH', 'SellerSKU': 'Schwarz-XL-100', 'Title': 'Einweghandschuhe Schwarz XL 100 Stück Nitrilhandschuhe puderfrei, Handschuhe Einweg latexfrei, strapazierfähige reißfeste Gummihandschuhe, Einweghandschuhe für Kochen, Reinigen, Tattoo & Nagelpflege', 'IsGift': 'false', 'ConditionSubtypeId': 'New', 'IsTransparency': False, 'QuantityOrdered': 2, 'PromotionDiscountTax': {'CurrencyCode': 'EUR', 'Amount': '0.00'}, 'ConditionId': 'New', 'PromotionDiscount': {'CurrencyCode': 'EUR', 'Amount': '0.00'}, 'OrderItemId': '37561007375002'}, {'ProductInfo': {'NumberOfItems': '1'},

In [30]:
# Order
order_id = jsonpath(order, "$._id")[0]
earliestDeliveryDate = jsonpath(order, "$.order.EarliestDeliveryDate")
earliestShipDate = jsonpath(order, "$.order.EarliestShipDate")[0]
salesChannel = jsonpath(order, "$.order.SalesChannel")[0]
orderStatus = jsonpath(order, "$.order.OrderStatus")[0]
latestShipDate = jsonpath(order, "$.order.LatestShipDate")[0]
purchaseDate = jsonpath(order, "$.order.PurchaseDate")[0]
orderTotalAmount = jsonpath(order, "$.order.OrderTotal.Amount")[0]
orderTotalCurrencyCode = jsonpath(order, "$.order.OrderTotal.CurrencyCode")[0]
lastUpdateDate = purchaseDate = jsonpath(order, "$.order.LastUpdateDate")[0]

# Order Items
titles = jsonpath(order, "$.items.OrderItems[*].Title")
asins = jsonpath(order, "$.items.OrderItems[*].ASIN")
sellerSKUs = jsonpath(order, "$.items.OrderItems[*].SellerSKU")
ItemPriceCurrencyCode = jsonpath(order, "$.items.OrderItems[*].ItemPrice.CurrencyCode")
ItemPriceAmount = jsonpath(order, "$.items.OrderItems[*].ItemPrice.Amount")
shippingPriceAmounts = jsonpath(order, "$.items.OrderItems[*].ShippingPrice.Amount")
shippingPriceCurrencyCodes = jsonpath(order, "$.items.OrderItems[*].ShippingPrice.CurrencyCode")
isTransparency = jsonpath(order, "$.items.OrderItems[*].IsTransparency")
quantityOrdered = jsonpath(order, "$.items.OrderItems[*].QuantityOrdered")
titles = jsonpath(order, "$.items.OrderItems[*].Title")


## 订单视图

In [31]:
order_dict = {
    "Order": {
        "order_id": jsonpath(order, "$._id")[0],
        "earliestDeliveryDate": jsonpath(order, "$.order.EarliestDeliveryDate")[0],
        "earliestShipDate": jsonpath(order, "$.order.EarliestShipDate")[0],
        "salesChannel": jsonpath(order, "$.order.SalesChannel")[0],
        "orderStatus": jsonpath(order, "$.order.OrderStatus")[0],
        "latestShipDate": jsonpath(order, "$.order.LatestShipDate")[0],
        "purchaseDate": jsonpath(order, "$.order.PurchaseDate")[0],
        "orderTotal": {
            "amount": jsonpath(order, "$.order.OrderTotal.Amount")[0],
            "currencyCode": jsonpath(order, "$.order.OrderTotal.CurrencyCode")[0]
        },
        "lastUpdateDate": jsonpath(order, "$.order.LastUpdateDate")[0]
    },
    "OrderItems": []
}

# 遍历订单中的每个订单项
for i in range(len(titles)):
    order_item = {
        "title": titles[i],
        "ASIN": asins[i],
        "sellerSKU": sellerSKUs[i],
        "ItemPrice": {
            "amount": ItemPriceAmount[i],
            "currencyCode": ItemPriceCurrencyCode[i]
        },
        "ShippingPrice": {
            "amount": shippingPriceAmounts[i] if shippingPriceAmounts else None,
            "currencyCode": shippingPriceCurrencyCodes[i] if shippingPriceCurrencyCodes else None
        },
        "isTransparency": isTransparency[i],
        "quantityOrdered": quantityOrdered[i]
    }
    order_dict["OrderItems"].append(order_item)


In [36]:
shippedOrders = list(orders_collection.find({"order.OrderStatus": "Shipped"}))
unshippedOrders = list(orders_collection.find({"order.OrderStatus": "Unshipped"}))

In [46]:
# 聚合管道，用于计算每个产品的寄出数量
pipeline = [
    {
        "$unwind": "$items.OrderItems"
    },
    { "$match" : { "order.OrderStatus" : "Shipped" } },
    {
        "$group": {
            "_id": "$items.OrderItems.SellerSKU",
            "totalQuantityShipped": {"$sum": "$items.OrderItems.QuantityShipped"}
        }
    }
]

results = orders_collection.aggregate(pipeline)
# 打印结果
for result in results:
    print(result)  #             "sku": "$items.OrderItems.SellerSKU",

{'_id': 'Meditrade 4471 Abe Fixiervlies', 'totalQuantityShipped': 1}
{'_id': 'MaiMed-Pad-unsteril 20x20cm', 'totalQuantityShipped': 0}
{'_id': 'HMMD-77040', 'totalQuantityShipped': 2}
{'_id': 'HGNI100S-XL', 'totalQuantityShipped': 9}
{'_id': 'FBM-Nasenverband 20 Stück-75115', 'totalQuantityShipped': 21}
{'_id': 'Blau-M-100', 'totalQuantityShipped': 3}
{'_id': 'Schwarz-XL-100', 'totalQuantityShipped': 30}
{'_id': 'Reinex Fugenreiniger 500 ml', 'totalQuantityShipped': 3}
{'_id': 'Lifemed Kühlspray 3er  300 ml', 'totalQuantityShipped': 1}
{'_id': 'Weiß-L-100', 'totalQuantityShipped': 1}
{'_id': 'HANS-751416D-5', 'totalQuantityShipped': 9}
{'_id': '2*Ungezieferköder 4-er', 'totalQuantityShipped': 4}
{'_id': 'HGBKPPPE100', 'totalQuantityShipped': 1}
{'_id': 'CB-YRCL-XUXH', 'totalQuantityShipped': 2}
{'_id': 'HMD-77590', 'totalQuantityShipped': 6}
{'_id': 'Dr. Becher Grillreiniger 5 l', 'totalQuantityShipped': 1}
{'_id': 'Reinex Wespenspray', 'totalQuantityShipped': 7}
{'_id': 'HMMD-77080', 

In [122]:
from datetime import datetime, timedelta
days_ago = 10
DATETIME_PATTERN = '%Y-%m-%d'
start_date = datetime.now() - timedelta(days=days_ago-1)
# 聚合管道，用于统计每个产品每天的寄出数量
pipeline = [
    {
        "$unwind": "$items.OrderItems"
    },
    {
        "$project": {
            "_id": 0,
            "orderDate": {"$dateFromString": {"dateString": "$order.PurchaseDate"}},
            "asin": "$items.OrderItems.ASIN",
            "quantityOrdered": "$items.OrderItems.QuantityOrdered",
            "quantityShipped": "$items.OrderItems.QuantityShipped",
            "sellerSKU": "$items.OrderItems.SellerSKU",
            "fulfillmentChannel": "$order.FulfillmentChannel",
            "title": "$items.OrderItems.Title"
        }
    },
    {
        "$match": {
            "orderDate": {"$gte": start_date}, 
            "fulfillmentChannel": "MFN"
        }
    },
    {
        "$group": {
            "_id": {
                "date": {"$dateToString": {"format": "%Y-%m-%d", "date": "$orderDate"}},
                "asin": "$asin"
            },
            "totalQuantityOrdered": {"$sum": "$quantityOrdered"},
            "totalQuantityShipped": {"$sum": "$quantityShipped"},
            "sellerSKU": {"$first": "$sellerSKU"},
            "title": {"$first": "$title"},
        }
    },
    {
        "$group": {
            "_id": "$_id.date",
            "purchaseDate": {"$first": "$_id.date"},
            "dailyShipments": {
                "$push": {
                   "asin": "$_id.asin",
                   "sellerSKU": "$sellerSKU",
                    "totalQuantityShipped": "$totalQuantityShipped",
                    "totalQuantityOrdered": "$totalQuantityOrdered",
                    "title": "$title",
                }
            }
        }
    },

    {
        "$sort": {"_id": -1}
    }
]
# 执行聚合操作
results = orders_collection.aggregate(pipeline)

# 打印结果
for result in results:
    print(result)

{'_id': '2024-04-21', 'purchaseDate': '2024-04-21', 'dailyShipments': [{'asin': 'B0CYH6S9RR', 'sellerSKU': '7L-7PPU-H005', 'totalQuantityShipped': 0, 'totalQuantityOrdered': 1, 'title': '10 Stück Spritze, 50ML Wiederverwendbare Spritze, Dosierspritze Ohne Nadel, Werkzeuge für Flüssigkeitsmaß Spritzen, Futterspritze, für Labor,Tierfütterung,Pflanzenbewässerung (50ml)'}, {'asin': 'B096MRHSW1', 'sellerSKU': 'Descolind Pure Intensive Cream 500ml', 'totalQuantityShipped': 0, 'totalQuantityOrdered': 1, 'title': 'DESCOLIND® PURE Intensive Cream 500 ml Spenderflasche Parfümfreie Intensiv-Pflegecreme'}]}
{'_id': '2024-04-20', 'purchaseDate': '2024-04-20', 'dailyShipments': [{'asin': 'B0CYH6S9RR', 'sellerSKU': '7L-7PPU-H005', 'totalQuantityShipped': 0, 'totalQuantityOrdered': 2, 'title': '10 Stück Spritze, 50ML Wiederverwendbare Spritze, Dosierspritze Ohne Nadel, Werkzeuge für Flüssigkeitsmaß Spritzen, Futterspritze, für Labor,Tierfütterung,Pflanzenbewässerung (50ml)'}, {'asin': 'B0CXXSGH74', 's